In [13]:
import pandas as pd
import numpy as np

prices_df = pd.read_csv('../Data assignment 1/prices_merged_df_output.csv', delimiter=',')

balUPVec = np.array(prices_df['BalancingPriceUpEUR'].values)
balDOWNVec = np.array(prices_df['BalancingPriceDownEUR'].values)
DAVec = np.array(prices_df['DA_PriceEUR'].values)

features_df = pd.read_csv('../Data assignment 1/merged_df_output.csv', delimiter=',')


actualWindproduction = np.array(features_df[' Total Wind Production Active Power'].values)


prices_df["HourDK"] = pd.to_datetime(prices_df["HourDK"])
Start_Observation = pd.to_datetime("2022-01-01 00:00:00")
End_Observation = pd.to_datetime("2022-01-02 00:00:00")
condition = (prices_df["HourDK"]>= Start_Observation) & (prices_df["HourDK"]< End_Observation)

prices_df  = prices_df[condition]

#randomprd = np.random.uniform(0.0, 30000.0, size=(24, 1))
#randomprd = np.round(randomprd, 2)

p_t = np.array([[22929.91751781],[13798.32478529],[ 3159.61283307],[ 2652.97866687],[ 6783.56761362],[11862.40683384],[ 5137.49087812],[27969.98682884],[21640.25680502],[13397.39663406],[13476.69929167],[ 2417.31715015],[26835.84184161],[14850.89123478],[  596.40330431],[26060.28463496],[20556.61807972],[ 9065.0593075 ],[27233.10392548],[ 9400.00567775],[22258.30389085],[  735.02622986],[ 2299.33859652],[19603.72469228]])

p_t = np.round(p_t, 2)









[[22929.91751781]
 [13798.32478529]
 [ 3159.61283307]
 [ 2652.97866687]
 [ 6783.56761362]
 [11862.40683384]
 [ 5137.49087812]
 [27969.98682884]
 [21640.25680502]
 [13397.39663406]
 [13476.69929167]
 [ 2417.31715015]
 [26835.84184161]
 [14850.89123478]
 [  596.40330431]
 [26060.28463496]
 [20556.61807972]
 [ 9065.0593075 ]
 [27233.10392548]
 [ 9400.00567775]
 [22258.30389085]
 [  735.02622986]
 [ 2299.33859652]
 [19603.72469228]]
